# Pick sample and visualize

In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
from IPython.display import display
from ipyfilechooser import FileChooser
import os
from data_utils.helpers import visualize_pointcloud
from data_utils.ModelNetDataLoader import pc_normalize

In [2]:
# Create and display a FileChooser widget
fc = FileChooser('/home/hoang/Dataset/')
# Set a file filter patern
fc.filter_pattern = '*.txt'
display(fc)

# Print the selected filename
print(fc.selected_filename)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


FileChooser(path='/home/hoang/Dataset', filename='', title='', show_hidden=False, select_desc='Select', change…

None


In [27]:
sample_points_path = os.path.join(fc.selected_path, fc.selected_filename)
visualize_pointcloud(sample_points_path, stride=10, flip_axis=-1, postprocess_fn=pc_normalize)

# Run model prediction

This model is trained with these hyperparams:

```
batch_size=32
num_category=42
epoch=200
num_point=6000
optimizer='Adam'
learning_rate=0.001
decay_rate=0.0001
```

Model training results:
Achieves 93.69% Test Accuracy


![](log/classification/pointnet2_cls_ssg_normal_42c/trainresults.png)

In [6]:
MODEL_NAME = 'pointnet2_cls_ssg'
CKPT_PATH = 'log/classification/pointnet2_cls_ssg_normal_42c/checkpoints/best_model.pth'
NUM_CLASS = 42
USE_NORMALS = True
DEVICE='cuda'

In [7]:
import importlib
import sys
import torch
from torchsummary import summary

# import model class
sys.path.append(os.path.join('models'))
checkpoint = torch.load(CKPT_PATH)

# get model by number of classes (categories)
model = importlib.import_module(MODEL_NAME)
classifier = model.get_model(NUM_CLASS, normal_channel=USE_NORMALS)

# load pretrained weights
classifier.load_state_dict(checkpoint['model_state_dict'])
classifier = classifier.eval()

# use summary to view model structure
summary(classifier, (6, 5000), col_names=['input_size', 'output_size', 'num_params', 'kernel_size']) # (xyzn, num_points)
None

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
├─PointNetSetAbstraction: 1-1            [-1, 3, 5000]             [-1, 3, 512]              --                        --
|    └─ModuleList: 2                     []                        []                        --                        --
|    |    └─Conv2d: 3-1                  [-1, 6, 32, 512]          [-1, 64, 32, 512]         448                       [6, 64, 1, 1]
|    └─ModuleList: 2                     []                        []                        --                        --
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 512]         [-1, 64, 32, 512]         128                       [64]
|    └─ModuleList: 2                     []                        []                        --                        --
|    |    └─Conv2d: 3-3                  [-1, 64, 32, 512]         [-1, 64, 32, 512]         4,160                     [64,

/home/hoang/miniconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning:

Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)



In [8]:
with open('data/modelnet40_withFootLast/modelnet40_shape_names.txt', 'r') as file:
    CLASSES = file.read().split()

print("# Classes:", len(CLASSES))
print(', '.join(CLASSES))

# Classes: 42
piano, bottle, bench, wardrobe, cup, night_stand, keyboard, curtain, lamp, tent, stairs, bookshelf, sofa, tv_stand, dresser, flower_pot, range_hood, monitor, toilet, laptop, vase, cone, door, radio, bathtub, desk, mantel, guitar, plant, car, airplane, person, table, foot, glass_box, sink, stool, chair, bed, last, xbox, bowl


In [10]:
import torch
import numpy as np
from data_utils.ModelNetDataLoader import pc_normalize

# read sample point cloud data into numpy array for prediction
points = np.loadtxt(sample_points_path, delimiter=',')
points[:, 0:3] = pc_normalize(points[:, 0:3]) # Normalize

# Prediction
with torch.no_grad():
    inputs = torch.tensor(points, dtype=torch.float)[None,].transpose(1,2).to(DEVICE) # Put on tensor
    logits, _ = classifier(inputs) # Get logits
    pred_idx = logits.data.max(1)[1].item()
    pred = CLASSES[pred_idx]
    
print("Model prediction:", pred.upper())

Model prediction: FOOT


In [11]:
import torch

torch.softmax(logits, dim=1).sort()

torch.return_types.sort(
values=tensor([[1.8098e-09, 2.3995e-08, 3.0905e-08, 3.1353e-08, 3.2265e-08, 3.7555e-08,
         6.2144e-08, 1.0883e-07, 1.0955e-07, 1.0967e-07, 1.4841e-07, 1.7830e-07,
         1.8773e-07, 2.9822e-07, 3.4201e-07, 4.4716e-07, 8.4089e-07, 8.6250e-07,
         1.0215e-06, 1.1302e-06, 1.2052e-06, 1.6201e-06, 2.0475e-06, 2.1647e-06,
         2.2308e-06, 2.2503e-06, 2.2615e-06, 3.6610e-06, 3.9793e-06, 5.9758e-06,
         7.5539e-06, 7.7524e-06, 1.0482e-05, 1.8142e-05, 2.3434e-05, 3.5602e-05,
         6.1791e-05, 6.6092e-05, 1.2869e-04, 4.7834e-04, 5.8420e-04, 9.9854e-01]],
       device='cuda:0'),
indices=tensor([[ 4, 17,  1, 22, 26,  5, 14, 32,  3, 15, 23, 31, 20, 25, 19, 29,  6, 13,
         18, 28, 35, 38, 40,  8, 34, 27, 11, 16, 24, 10,  7, 36,  0, 30, 21, 37,
         12, 41,  2,  9, 39, 33]], device='cuda:0'))

In [12]:
logits.shape

torch.Size([1, 42])

In [13]:
CLASSES[27]

'guitar'